In [1]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi

## CREATE GENETIC DATA (and clean/process/edit)

In [4]:
# Create genetic data and write to disk
bnm_mt = hl.balding_nichols_model(3, 100, 1000)
bnm_mt.write("balding_nichols_3_100_1000.mt")

2020-07-27 10:17:45 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 100 samples, and 1000 variants...
2020-07-27 10:17:45 Hail: INFO: Coerced sorted dataset
2020-07-27 10:17:48 Hail: INFO: wrote matrix table with 1000 rows and 100 columns in 8 partitions to balding_nichols_3_100_1000.mt


In [5]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

# Turn MatrixTable into Table

ht = mt.localize_entries("ent", "sample")

2020-07-27 10:19:25 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


## Grouping and NDArray methods from Tim and Dan

In [42]:
# Functions for operating with Tables of ndarrays in Hail (from Tim)

from hail.expr import Expression, ExpressionException, \
    expr_float64, expr_call, expr_any, expr_numeric, expr_array, \
    expr_locus, \
    analyze, check_entry_indexed, check_row_indexed, \
    matrix_table_source, table_source

# Only groups by rows, NOT COLUMNS
def matrix_table_to_table_of_ndarrays(field, group_size, tmp_path = '/tmp/nd_table.ht'):
    """

    The returned table has two fields: 'row_group_number' and 'ndarray'.

    Examples
    --------
    >>> ht = matrix_table_to_table_of_ndarrays(mt.GT.n_alt_alleles(), 100)

    Parameters
    ----------
    field
    group_size
    tmp_path

    Returns
    -------

    """
    mt = matrix_table_source('matrix_table_to_table_of_ndarrays/x', field)
    mt = mt.select_entries(x = field)
    ht = mt.localize_entries(entries_array_field_name='entries')
    # now ht.entries is an array of structs with one field, x

    # we'll also want to mean-impute/variance-normalize/etc here
    ht = ht.select(xs = ht.entries.map(lambda e: e['x']))
    # now ht.xs is an array of float64

    # now need to produce groups of G
    ht = ht.add_index()
    ht = ht.group_by(row_group_number= hl.int32(ht.idx // group_size)) \
        .aggregate(ndarray=hl.nd.array(hl.agg.collect(ht.xs)))
    # may require a .T on ndarray

    return ht.checkpoint(tmp_path, overwrite=True)

def chunk_ndarray(a, group_size):
    """Chunks a NDarray along the first axis in chunks of `group_size`.
    Parameters
    ----------
    a
    group_size
    -------

    """
    n_groups = a.shape[0] // group_size
    groups = []
    for i in range(a.shape[0] // group_size):
        start = i * group_size
        end = (i + 1) * group_size
        groups.append(a[start:end, :])
    return groups


# Concatenate the ndarrays with a blocked Table
def concatBlocked(A):
    blocks = A.ndarray.collect()
    big_mat = np.concatenate(blocks, axis=0)
    ht = ndarray_to_table([big_mat])
    
    block_shape = blocks[0].shape
    
    tup = ht.ndarray.collect()[0].shape
    assert (tup == (len(blocks) * block_shape[0], block_shape[1]))
    
    return ht

def concatToNumpy(A):
    blocks = A.ndarray.collect()
    big_mat = np.concatenate(blocks, axis=0)
    return big_mat

# takes ndarray, possibly already in chunks/groups and converts to a 
# the input array should always be an array of the chunks
# meaning, if there is no chunking of arr at all, one should input [arr]
# Hail Table with subarray chunks as entries
# def ndarray_to_table(chunked_arr, num_blocks):
#     ht = hl.utils.range_table(num_blocks)
#     structs = [hl.struct(row_group_number = idx, ndarray = block) 
#                for idx, block in enumerate(chunked_arr)] # this line has a bug
#     ht = hl.Table.parallelize(structs) # or this line
#     ht = ht.key_by('row_group_number') # or this line
#     return ht


def ndarray_to_table(chunked_arr):
    structs = [hl.struct(row_group_number = idx, ndarray = block)
               for idx, block in enumerate(chunked_arr)]
    ht = hl.Table.parallelize(structs)
    ht = ht.key_by('row_group_number')
    return ht



# function to multiply two blocks, given the two blocks
# returns struct in form of array but not ndarray, includes the shape in the struct
# to change the result product directly back into a ndarray we need to use from_column_major
def block_product(left, right):
    product = left @ right
    n_rows, n_cols = product.shape
    return hl.struct(
        shape=product.shape,
        block=hl.range(hl.int(n_rows * n_cols)).map(
            lambda absolute: product[absolute % n_rows, absolute // n_rows]))

# takes in output of block_product
def block_aggregate(prod):
    shape = prod.shape
    block = prod.block
    return hl.nd.from_column_major(
        hl.agg.array_sum(block),
        hl.agg.take(shape, 1)[0])

# returns flat array
def to_column_major(ndarray):
    n_rows, n_cols = ndarray.shape
    return hl.range(hl.int(n_rows * n_cols)).map(
        lambda absolute: ndarray[absolute % n_rows, absolute // n_rows])

# hl.nd.from_column_major(thing.the_sum, thing.the_shape)

## Blanczos Algorithm

In [7]:
def makeData(model_input, group_size):
    mt = hl.balding_nichols_model(*model_input)
    mt.write("balding_nichols_test.mt")
    mt = hl.read_matrix_table("balding_nichols_test.mt")
    mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 
    table = mt.localize_entries("ent", "sample")
    table = matrix_table_to_table_of_ndarrays(mt.n_alt, group_size, tmp_path='/tmp/test_table.ht')
#     table = table.key_by(hl.int32(table.row_group_number))
    return table
    
data = makeData((3, 100, 1000), 4)


2020-07-27 10:19:46 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 100 samples, and 1000 variants...
2020-07-27 10:19:47 Hail: INFO: Coerced sorted dataset
2020-07-27 10:19:48 Hail: INFO: wrote matrix table with 1000 rows and 100 columns in 8 partitions to balding_nichols_test.mt
2020-07-27 10:19:58 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-27 10:20:01 Hail: INFO: wrote table with 250 rows in 8 partitions to /tmp/test_table.ht


In [43]:
(n, m) = (100, 1000)
k = 50
l = k + 2
q = 0

G = hl.nd.array(np.random.normal(0, 1, (n,l)))

In [61]:
# Algorithm step: multiplying H0 = A @ G

# METHOD
# Multiply a row-blocked matrix by a local non-blocked matrix
# First step of algorithm

# usage:
# assumes blocks in blocked matrix are named ndarray
# A is a table, B is a Hail ndarray
def matmul_rowblocked_nonblocked(A, B):
    temp = A.annotate_globals(mat = B)
    temp = temp.annotate(ndarray = temp.ndarray @ temp.mat)
#     temp = temp.annotate(prod = block_product(temp.ndarray, temp.mat))
#     temp = temp.annotate(ndarray = hl.nd.from_column_major(temp.prod.block, temp.prod.shape))
    temp = temp.select(temp.ndarray)
    temp = temp.drop(temp.mat)
    return temp

H0 = matmul_rowblocked_nonblocked(data, G)

# Algorithm step: intermediate operation of multiplying At @ (A @ G) = At @ H0

# METHOD
# Multiply a column-blocked matrix by a row-blocked matrix 
# as a blockmatrix multiplcation and then sum
# Second step of algorithm

# usage:
# pass in matrix A normally, blocked in rows - this specifically expects A to need to be transposed
# assumes blocks in blocked matrix are named ndarray
# A and B are both tables
def matmul_colblocked_rowblocked(A, B):
    temp = A.transmute(prod = block_product(A.ndarray.transpose(), B[A.row_group_number].ndarray))
    result_arr_sum = temp.aggregate(block_aggregate(temp.prod))
    return result_arr_sum

G1 = matmul_colblocked_rowblocked(data, H0)


def matmul_nonblocked_rowblocked(B, A):
    pass


<class 'numpy.ndarray'>


In [62]:
# Algorithm step: perform QR decomposition of Hq and compute T = Q^T @ A
# Perform QR decomposition of a row-blocked matrix
# Third and fourth step of algorithm

def computeNextH(H, A):
    nextG = matmul_colblocked_rowblocked(A, H)
    return matmul_rowblocked_nonblocked(A, nextG)


def processH(H, A):
    
    # perform QR decomposition on unblocked version of H
    arr_H = concatToNumpy(H)
    #assert (arr_H.shape == (m, (q+1)*l))
    Q, R = np.linalg.qr(arr_H)
    assert(Q.shape == (m, (q+1)*l))
    
    # block Q's rows into the same number of blocks that A has
    num_blocks = A.count()
    tup = hl.eval(Q.shape)
    group_size_Q = tup[0] // num_blocks
    #assert group_size_Q * num_blocks == m
    blocked_Q_table = ndarray_to_table(chunk_ndarray(Q, group_size_Q))
    
    T = matmul_colblocked_rowblocked(blocked_Q_table, A)
    return T, Q, blocked_Q_table

# Algorithm step: compute SVD of T such that T = USW^T
# 5th step of algorithm

def processT(T):
    arr_T = T #T.collect()[0]
    print('T.shape', arr_T.shape)
    U, S, W = np.linalg.svd(arr_T, full_matrices=False)
    print('u shape:', U.shape, 's shape:', S.shape, 'w shape:', W.shape)
    assert(U.shape == (l, (q+1)*l))
    assert(S.shape == ((q+1)*l,))
    assert(W.shape == ((q+1)*l, n))
    return U, S, W

# Algorithm step: multiply V = Q @ W
# 6th step of algorithm and last step

def computeV(U, S, W, Q):
#     table_W = ndarray_to_table([W])
#     print(Q.count(), Q.collect()[0].ndarray.shape)
#     print(table_W.collect()[0].ndarray.shape)
    V = matmul_rowblocked_nonblocked(Q, U)
    return U, S, V

In [66]:
T, Q, table_Q = processH(H0, data)
print('Q shape', Q.shape)

2020-07-27 13:21:28 Hail: INFO: Coerced sorted dataset


<class 'numpy.ndarray'>
Q shape (1000, 52)


In [68]:
# print(mat_T.ndarray.collect()[0])
# print(mat_T.ndarray.collect()[0].shape)
u, s, w = processT(T)
U, S, V = computeV(u, s, w, table_Q)

T.shape (52, 100)
u shape: (52, 52) s shape: (52,) w shape: (52, 100)


In [69]:
V.ndarray.show()

2020-07-27 13:22:48 Hail: INFO: Coerced sorted dataset


+------------------+
| row_group_number |
+------------------+
|            int32 |
+------------------+
|                0 |
|                1 |
|                2 |
|                3 |
|                4 |
|                5 |
|                6 |
|                7 |
|                8 |
|                9 |
|               10 |
|               11 |
|               12 |
|               13 |
|               14 |
|               15 |
|               16 |
|               17 |
|               18 |
|               19 |
|               20 |
+------------------+

+-----------------------------------------------------------------------------------+
| ndarray                                                                           |
+-----------------------------------------------------------------------------------+
| ndarray<float64, 2>                                                               |
+-----------------------------------------------------------------------------------+
| ndarray{shape=(4, 52), data=[[-0.033949023453449606, -0.032125878714415995, -0... |
| ndarray{shape=(4, 52), data=[[-0.023443964523316527, -0.03319053523618848, -0.... |
| ndarray{shape=(4, 52), data=[[-0.04401248952979731, -0.01219826006487324, -0.0... |
| ndarray{shape=(4, 52), data=[[-0.04401907669618028, -0.05275147934333333, -0.0... |
| ndarray{shape=(4, 52), data=[[-0.008664893315195151, -0.05158904419217613, -0.... |
| ndarray{shape=(4, 52), data=[[-0.04501854245815365, -0.040123266776387764, -0.... |
| ndarray{shape=(4, 52), data=[[-0.015605198526345912, -0.027414411566095483, -0... |
| ndarray{shape=(4, 52), data=[[-0.035588863300375195, -0.01962261632726966, -0.... |
| ndarray{shape=(4, 52), data=[[-0.025686721361287662, -0.03336602257931119, -0.... |
| ndarray{shape=(4, 52), data=[[-0.026612946567584627, -0.0295814644494395, -0.0... |
| ndarray{shape=(4, 52), data=[[-0.02268548764109173, -0.03312252882564441, -0.0... |
| ndarray{shape=(4, 52), data=[[-0.03400620652400482, -0.022559517163077882, -0.... |
| ndarray{shape=(4, 52), data=[[-0.016364181049060292, -0.025101147801952634, -0... |
| ndarray{shape=(4, 52), data=[[-0.04802432283159779, -0.0187995771303401, -0.02... |
| ndarray{shape=(4, 52), data=[[-0.024080014222363798, -0.01712132756938764, -0.... |
| ndarray{shape=(4, 52), data=[[-0.01747104760562928, -0.014245993782200851, -0.... |
| ndarray{shape=(4, 52), data=[[-0.015855938518489862, -0.021333992795818345, -0... |
| ndarray{shape=(4, 52), data=[[-0.05222417792270041, -0.009688654833089168, -0.... |
| ndarray{shape=(4, 52), data=[[-0.05049348224254874, -0.029085470630862767, -0.... |
| ndarray{shape=(4, 52), data=[[-0.008198119248449933, -0.006733715599839872, -0... |
| ndarray{shape=(4, 52), data=[[-0.021199187862050838, -0.021938648927899873, -0... |
+-----------------------------------------------------------------------------------+
showing top 21 rows

In [132]:
# ALL TRANSPOSED FROM ABOVE IMPLEMENTATION

def hwe_normalize(call_expr):
    mt = call_expr._indices.source
    mt = mt.select_entries(__gt=call_expr.n_alt_alleles())
    mt = mt.annotate_rows(__AC=hl.agg.sum(mt.__gt),
                          __n_called=hl.agg.count_where(hl.is_defined(mt.__gt)))
    mt = mt.filter_rows((mt.__AC > 0) & (mt.__AC < 2 * mt.__n_called))

    n_variants = mt.count_rows()
    if n_variants == 0:
        raise FatalError("hwe_normalized: found 0 variants after filtering out monomorphic sites.")

    mt = mt.annotate_rows(__mean_gt=mt.__AC / mt.__n_called)
    mt = mt.annotate_rows(
        __hwe_scaled_std_dev=hl.sqrt(mt.__mean_gt * (2 - mt.__mean_gt) * n_variants / 2))
    mt = mt.unfilter_entries()

    normalized_gt = hl.or_else((mt.__gt - mt.__mean_gt) / mt.__hwe_scaled_std_dev, 0.0)
    return normalized_gt

#Blanczos paper error bound 4.3
def blanczosError(U, S, V, m, n, k, q, A, k1th_sing_val):
    norm_diff = np.linalg.norm(A - U @ S @ V.transpose())
    bound = 100 * l * (((m-k)/l) ** (1/(4*q + 2))) * k1th_sing_val
    return norm_diff <= bound

def blanczosSVD(A, m, n, k, l, q):

    assert l > k
    assert (q+1)*l <= (m - k)
    assert m <= n

    G = np.random.normal(0, 1, (l, m))
    R = G @ A
    #AtA = A.transpose() @ A
    listR = [R]
    for i in range(0, q):
        Ri = (listR[i] @ A.transpose()) @ A
        listR.append(Ri)
        R = np.concatenate((R, Ri), axis=0)

    (Q, S) = np.linalg.qr(R.transpose())
    assert(Q.shape == (n, (q+1)*l))
    assert(S.shape == ((q+1)*l, (q+1)*l))

    T = A @ Q
    assert(T.shape == (m, (q+1)*l))
    
    (Tu, Ts, Tw) = np.linalg.svd(T, full_matrices=False)
    assert(Tu.shape, (m, (q+1)*l))
    assert(Ts.shape == ((q+1)*l,))
    assert(Tw.shape == ((q+1)*l, (q+1)*l))
    
    
    V = Q @ Tw
    
    bound = blanczosError(Tu, np.diag(Ts), V, m, n, k, q, A, Ts[k])
    print("Satisfies Blanczos error bound equation 4.3: ", bound)

    return (Tu, Ts, V)


mt = hl.balding_nichols_model(3, 10000, 1000)
norm_gt = hwe_normalize(mt.GT)
np_matrix = hl.linalg.BlockMatrix.from_entry_expr(norm_gt).to_numpy()

npA = np.asmatrix(np_matrix)
print(npA.shape)
npm, npn = npA.shape
npk = 50
npl = npk + 2
npq = 0

(blanczosU, blanczosS, blanczosV) = blanczosSVD(npA, npm, npn, npk, npl, npq)

2020-07-24 17:18:34 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 10000 samples, and 1000 variants...
2020-07-24 17:18:36 Hail: INFO: Coerced sorted dataset
2020-07-24 17:18:40 Hail: INFO: Wrote all 3 blocks of 1000 x 10000 matrix with block size 4096.


(1000, 10000)
Satisfies Blanczos error bound equation 4.3:  True


In [134]:
blanczosV.shape

(10000, 52)